In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from PIL import Image
from sklearn.decomposition import PCA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import os

# Define the paths to your zip files on Google Drive
beds_zip = '/content/drive/My Drive/beds.zip'
sofa_zip = '/content/drive/My Drive/sofa.zip'
chairs_zip = '/content/drive/My Drive/chairs.zip'
tables_zip = '/content/drive/My Drive/tables.zip'
shelving_zip = '/content/drive/My Drive/shelving.zip'
wardrobe_zip = '/content/drive/My Drive/wadrobe.zip'

# Unzip each file into its corresponding folder
with zipfile.ZipFile(beds_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/beds')

with zipfile.ZipFile(sofa_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/sofa')

with zipfile.ZipFile(chairs_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/chairs')

with zipfile.ZipFile(tables_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/tables')

with zipfile.ZipFile(shelving_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/shelving')

with zipfile.ZipFile(wardrobe_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/wadrobe')


In [ ]:
beds_folder = '/content/beds'
sofa_folder = '/content/sofa'
chairs_folder = '/content/chairs'
tables_folder = '/content/tables'
shelving_folder = '/content/shelving'
wardrobe_folder = '/content/wadrobe'


IMAGE FOLDER EXTRACTION FROM DRIVE

---



In [ ]:
# Function to load images from a folder and create a DataFrame
def load_images(folder_path):
    image_paths = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            if image is not None:  # Check if image was loaded successfully
                image_paths.append(image_path)
    return pd.DataFrame({'image_path': image_paths})

# Load images from each folder
beds_df = load_images(beds_folder)
sofa_df = load_images(sofa_folder)
chairs_df = load_images(chairs_folder)
tables_df = load_images(tables_folder)
shelving_df = load_images(shelving_folder)
wadrobe_df = load_images(wardrobe_folder)


# Combine DataFrames
combined_df = pd.concat([beds_df, sofa_df, chairs_df, tables_df, shelving_df, wadrobe_df])
print(combined_df.shape)

In [ ]:
 from google.colab.patches import cv2_imshow

DUPLICATION REMOVAL

---



In [ ]:
# Remove duplicate images based on image content
def remove_duplicates(df):
    image_hashes = []
    unique_df = pd.DataFrame()
    for index, row in df.iterrows():
        image_path = row['image_path']
        image = cv2.imread(image_path)
        image_hash = hash(image.tobytes())
        if image_hash not in image_hashes:
            image_hashes.append(image_hash)
            unique_df = pd.concat([unique_df, row.to_frame().T], ignore_index=True)
    return unique_df

# Remove duplicates from the combined DataFrame
combined_df = remove_duplicates(combined_df)
print(combined_df.shape)

IMAGE PROCESSING

---



In [ ]:
def process_image(image_path, image_size=(80, 80)):
    try:
        img = Image.open(image_path).convert('RGB')  # Open and convert to RGB
        # Resize the image
        img = img.resize(image_size)
        img_array = np.array(img)  # Convert to a numpy array
        img_array = img_array.flatten()  # Flatten the 3D image to 1D
        return img_array
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

In [ ]:

# Apply the process_image function to the 'image_path' column of the DataFrame
combined_df['processed_image'] = combined_df['image_path'].apply(process_image)


In [ ]:
def color_edge_detection(image, image_size=(80,80)):
  try:
    img=img.resize(image_size)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (3,3), 0)
    # Apply Canny edge detection
    edges = cv2.Canny(gray, 50, 150)
    img_array=edges.flatten()


    return img_array

  except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None


combined_df['processed_edges'] = combined_df['image_path'].apply(process_image)


In [ ]:
combined_df = combined_df.applymap(lambda x: [x] if isinstance(x, int) else x)

In [ ]:
combined_df['processed_image']=combined_df['processed_image'].dropna()


In [ ]:
combined_df = combined_df[combined_df['processed_image'].apply(lambda x: not isinstance(x, float))]
new_df = pd.DataFrame(combined_df['processed_image'].to_list())


PCA

---



In [ ]:
combined_df['processed_edges']=combined_df['processed_edges'].dropna()


In [ ]:
combined_df = combined_df[combined_df['processed_edges'].apply(lambda x: not isinstance(x, float))]
new_df_edges = pd.DataFrame(combined_df['processed_edges'].to_list())


In [ ]:
new_df_edges

In [ ]:
from transformers import AutoImageProcessor, ViTMAEModel
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

image_processor = AutoImageProcessor.from_pretrained("facebook/vit-mae-base")
model = ViTMAEModel.from_pretrained("facebook/vit-mae-base")

inputs = image_processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

In [ ]:
from transformers import AutoImageProcessor, ViTMAEModel
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

image_processor = AutoImageProcessor.from_pretrained("facebook/vit-mae-base")
model = ViTMAEModel.from_pretrained("facebook/vit-mae-base")

inputs = image_processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state


In [ ]:
import torch
import torch.nn as nn


In [ ]:
class FeatureExtractor(nn.Module):

    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.model = ViTMAEModel.from_pretrained("facebook/vit-mae-base")

    def forward(self, inputs):
        inputs.pop('image_name', None)

        outputs = self.model(**inputs)
        return torch.sum(outputs.last_hidden_state, dim=1)

In [ ]:
from transformers import AutoProcessor, ViTMAEModel
import torch
import torch.nn as nn
from PIL import Image
import pandas as pd

# Define the FeatureExtractor class
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.model = ViTMAEModel.from_pretrained("facebook/vit-mae-base")

    def forward(self, inputs):
        inputs.pop('image_name', None)
        outputs = self.model(**inputs)
        return torch.sum(outputs.last_hidden_state, dim=1)

# Load the processor
processor = AutoProcessor.from_pretrained("facebook/vit-mae-base")

# Function to process each image and extract features
def extract_features_from_df(row, feature_extractor, processor):
    try:
        # Load the image from the path
        image = Image.open(row['image_path'])

        # Preprocess the image
        inputs = processor(images=image, return_tensors="pt")

        # Extract features
        with torch.no_grad():  # For inference
            features = feature_extractor(inputs)

        # Convert tensor to list (or numpy array) if you want to store it in a DataFrame
        return features.squeeze().tolist()

    except Exception as e:
        print(f"Error processing image {row['image_path']}: {e}")
        return None

# Example DataFrame with image paths
df = pd.DataFrame(combined_df['image_path'])
# Instantiate the FeatureExtractor
feature_extractor = FeatureExtractor()

# Apply the feature extraction to each row
df['features'] = df.apply(lambda row: extract_features_from_df(row, feature_extractor, processor), axis=1)

# Check the output DataFrame with extracted features
print(df.head())


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Error processing image /content/chairs/image_1698.jpg: Unsupported number of image dimensions: 2
Error processing image /content/tables/image_1619.jpg: Unsupported number of image dimensions: 2
Error processing image /content/tables/image_1717.jpg: Unsupported number of image dimensions: 2
                     image_path  \
0  /content/sofa/image_1257.jpg   
1   /content/sofa/image_396.jpg   
2  /content/sofa/image_1413.jpg   
3   /content/sofa/image_971.jpg   
4  /content/sofa/image_1305.jpg   

                                            features  
0  [-0.6886653900146484, -7.901733875274658, -6.1...  
1  [-6.35953950881958, 0.3591250777244568, 0.0132...  
2  [-8.57598876953125, 1.2275011539459229, -0.181...  
3  [-4.2629780769348145, 1.2356879711151123, -1.3...  
4  [-7.434243202209473, 0.06401626765727997, 1.43...  


In [ ]:
len(df.iloc[0,1])

768

In [ ]:
# Drop rows where 'features' column contains NaN or None
df = df.dropna(subset=['features'])

# Ensure 'features' column contains lists (or valid feature data)
df = df[df['features'].apply(lambda x: isinstance(x, list))]

# Convert the list of features into a new DataFrame (expand each list element into its own column)
features_df = pd.DataFrame(df['features'].to_list())

# Reset the index of both DataFrames to ensure unique indices
df = df.reset_index(drop=True)
features_df = features_df.reset_index(drop=True)

# Concatenate both DataFrames along the columns
df = pd.concat([df, features_df], axis=1)

# Check the updated DataFrame
print(df.head())


                     image_path  \
0  /content/sofa/image_1257.jpg   
1   /content/sofa/image_396.jpg   
2  /content/sofa/image_1413.jpg   
3   /content/sofa/image_971.jpg   
4  /content/sofa/image_1305.jpg   

                                            features         0         1  \
0  [-0.6886653900146484, -7.901733875274658, -6.1... -0.688665 -7.901734   
1  [-6.35953950881958, 0.3591250777244568, 0.0132... -6.359540  0.359125   
2  [-8.57598876953125, 1.2275011539459229, -0.181... -8.575989  1.227501   
3  [-4.2629780769348145, 1.2356879711151123, -1.3... -4.262978  1.235688   
4  [-7.434243202209473, 0.06401626765727997, 1.43... -7.434243  0.064016   

          2         3          4          5         6         7  ...  \
0 -6.199350  7.763186 -32.192436  -7.438576  3.239686  3.725930  ...   
1  0.013264 -5.996150 -24.487288  -8.854685  6.449086  2.317488  ...   
2 -0.181056 -1.323359 -27.304140  -4.040198  3.858768  2.606106  ...   
3 -1.352839 -0.813753 -19.529764 -21.539541 

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Standardize features for DBSCAN (important for distance-based clustering algorithms)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df['features'].to_list())  # Assuming 'features' contains the extracted features

# Apply DBSCAN clustering
dbscan = DBSCAN(eps=12.5, min_samples=5)  # Adjust 'eps' and 'min_samples' based on your data
df['cluster'] = dbscan.fit_predict(scaled_features)

# Check the cluster labels
print(df['cluster'].value_counts())


cluster
-1    8360
 0      28
 3      21
 1      17
 5      16
 8      13
 6      10
 2       7
 4       6
 7       5
Name: count, dtype: int64


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Assuming df is your DataFrame and 'features' contains the extracted features
# Replace 'df' with the actual DataFrame you are using

# Standardize features for DBSCAN (important for distance-based clustering algorithms)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df['features'].to_list())  # Convert 'features' column to list before scaling

# Define a range of eps values (from 10 to 20)
eps_values = np.arange(10, 21, 1)

# Initialize lists to store results
num_clusters = []
silhouette_scores = []

for eps in eps_values:
    # Fit DBSCAN model with the current eps
    dbscan = DBSCAN(eps=eps, min_samples=5)
    df['cluster'] = dbscan.fit_predict(scaled_features)  # Predict clusters and assign to DataFrame

    # Get the labels and calculate the number of clusters (ignoring noise points)
    labels = df['cluster']
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    num_clusters.append(n_clusters)

    # Calculate silhouette score if there's more than 1 cluster
    if n_clusters > 1:
        sil_score = silhouette_score(scaled_features, labels)
        silhouette_scores.append(sil_score)
    else:
        silhouette_scores.append(-1)  # Invalid silhouette score for single cluster or no clusters

# Plot the number of clusters vs. eps (Elbow plot)
plt.figure(figsize=(10, 6))
plt.plot(eps_values, num_clusters, marker='o', label='Number of Clusters')
plt.title('Elbow Plot for DBSCAN (Number of Clusters vs. eps)')
plt.xlabel('eps')
plt.ylabel('Number of Clusters')
plt.grid(True)
plt.legend()
plt.show()

# Plot the silhouette score vs. eps
plt.figure(figsize=(10, 6))
plt.plot(eps_values, silhouette_scores, marker='o', color='green', label='Silhouette Score')
plt.title('Silhouette Score vs. eps')
plt.xlabel('eps')
plt.ylabel('Silhouette Score')
plt.grid(True)
plt.legend()
plt.show()

# Apply DBSCAN with eps = 18
dbscan = DBSCAN(eps=18, min_samples=5)
df['cluster'] = dbscan.fit_predict(scaled_features)

# Check the cluster labels (number of samples in each cluster)
print(df['cluster'].value_counts())


In [ ]:
import random
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from PIL import Image

random_index = random.choice(df.index)
random_image_features = df.loc[random_index, 'features']
random_cluster = df.loc[random_index, 'cluster']


same_cluster_df = df[df['cluster'] == random_cluster]
cluster_features = np.array(same_cluster_df['features'].tolist())

similarities = cosine_similarity([random_image_features], cluster_features).flatten()

most_similar_indices = similarities.argsort()[-4:-1]

selected_image_path = df.loc[random_index, 'image_path']
similar_image_paths = same_cluster_df.iloc[most_similar_indices]['image_path'].tolist()

def display_images(image_paths, title):
    plt.figure(figsize=(10, 5))
    for i, img_path in enumerate(image_paths):
        img = Image.open(img_path)
        plt.subplot(1, len(image_paths), i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

display_images([selected_image_path], "Selected Image")

display_images(similar_image_paths, "3 Most Similar Images")
